# Loss and Accuracy over Epochs

In [ ]:
import glob
import matplotlib.pyplot as plt
import json
import os
from itertools import cycle


def loss_plot(xlim: (int, int) = None):
    # Use glob to find all json files in the specified directory
    file_list = glob.glob('results/metrics_*.json')

    # Create a color cycle iterator to assign unique colors
    color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

    # Plot the data
    plt.figure(figsize=(14, 7))

    # Initialize a dictionary to store colors for each label
    color_map = {}

    # Subplot for Loss
    plt.subplot(1, 2, 1)
    for file in file_list:
        # Extract model_name and label_name from the file name
        filename = os.path.basename(file)
        model_label = filename.replace('metrics_', '').rsplit('.', 1)[0]

        # Check if we have already assigned a color, otherwise get the next color
        if model_label not in color_map:
            color_map[model_label] = next(color_cycle)

        # Read json content
        with open(file, 'r') as f:
            data = json.load(f)

        epochs = list(range(1, len(data['train_loss']) + 1))
        plt.plot(epochs, data['train_loss'], label=f'{model_label}',
                 color=color_map[model_label])
        if len(data['val_acc']) > 0:
            plt.plot(epochs, data['val_loss'], linestyle='--', color=color_map[model_label])

    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xlim(xlim)

    # Subplot for Accuracy
    plt.subplot(1, 2, 2)
    for file in file_list:
        # Extract model_name and label_name from the file name
        filename = os.path.basename(file)
        model_label = filename.replace('metrics_', '').rsplit('.', 1)[0]

        # Read json content
        with open(file, 'r') as f:
            data = json.load(f)

        epochs = list(range(1, len(data['train_acc']) + 1))
        plt.plot(epochs, data['train_acc'], label=f'{model_label}',
                 color=color_map[model_label])
        if len(data['val_acc']) > 0:
            plt.plot(epochs, data['val_acc'], linestyle='--', color=color_map[model_label])

    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    # 添加自定义图例
    custom_lines = [plt.Line2D([0], [0], color='black', lw=2),
                    plt.Line2D([0], [0], color='black', lw=2, linestyle='--')]
    plt.figlegend(custom_lines, ['Train', 'Val'], loc='lower center', ncol=2)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.legend()
    plt.xlim(xlim)
    plt.show()


In [ ]:
loss_plot()

In [ ]:
import glob
import matplotlib.pyplot as plt
import json
import os
import numpy as np

start_epoch = 0

file_list = glob.glob('results/metrics_*.json')
for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)

    print(file)
    print(int(np.argmin(data['val_loss'][start_epoch:]))+start_epoch)
    print(int(np.argmax(data['val_acc'][start_epoch:]))+start_epoch)
    

# Show cropped images

In [ ]:
from VisionDatasets import ContactDataset
from PIL import Image, ImageDraw
import json
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torchvision
import numpy as np
from tqdm import tqdm

In [ ]:
paths_json = os.path.join(os.getcwd(), 'settings/paths.json')
with open(paths_json, 'r') as json_file:
    paths = json.load(json_file)

data = pd.read_csv(paths['silicone']['labels'], header=0).sample(n=12)

dataset = ContactDataset(
    images=data['HPC_Path'].tolist(),
    labels=data['GT'].to_numpy(),
    coords=list(zip(
        data['x'].astype(int),
        data['y'].astype(int))),
    jitter=False)

dataloader = DataLoader(
    dataset=dataset,
    batch_size=1024,
    pin_memory=True)

In [ ]:
width, height = 936, 702
merged_image = Image.new('RGB', (width, height))

draw = ImageDraw.Draw(merged_image)

small_image_width, small_image_height = 234, 234
x, y = 0, 0

for images, labels in dataloader:
    for image in images:
        image = torchvision.transforms.ToPILImage()(image)
        merged_image.paste(image, (x, y))
    
        x += small_image_width
        
        if x + small_image_width > width:
            x = 0
            y += small_image_height

merged_image.show()

# Time Plot

In [1]:
from itertools import product
import numpy as np
import pickle
import matplotlib.pyplot as plt

model_set = ['CustomCNN', 'EfficientNet']
label_set = ['MTurk']
model_combinations = list(product(model_set, label_set))
test_set = [37, 68, 71, 23, 22, 24, 25, 26, 27, 28, 29,
            46, 47, 38, 95, 104, 101, 92, 110, 45, 125, 134]

In [12]:
with open('labels/silicone_20231113_040138.pkl', 'rb') as file:
    results = pickle.load(file)


for set_name in test_set:
    data = {
        'gt': None,
        'pred': {},
        'binary_predictions': {}
    }
    # Process the data for each model and label combination
    for model_name, label_name in model_combinations:
        pred, gt = results[(model_name, label_name, set_name)].values()
        binary_predictions = (pred > 0.5).astype(int)

        # Store ground truth only once as it's the same for all combinations
        if data['gt'] is None:
            data['gt'] = gt

        # Store predictions and binary predictions
        data['pred'][f'{model_name}-{label_name}'] = pred
        data['binary_predictions'][f'{model_name}-{label_name}'] = binary_predictions

    
    colors = plt.cm.get_cmap('Set1', len(data['pred']))

    # Convert indices to time (assuming a sampling rate of 30Hz)
    sampling_rate_hz = 30
    time = np.arange(len(data['gt'])) / sampling_rate_hz

    # Plotting
    plt.figure(figsize=(10, 6))

    # Plot ground truth
    plt.plot(time, data['gt'], label='Ground Truth', color='black')

    # Plot predictions and binary predictions
    for i, key in enumerate(data['pred']):
        color = colors(i)
        plt.plot(time, data['pred'][key], label=f'Pred - {key}', alpha=0.7, linestyle='--', color=color)
        plt.plot(time, data['binary_predictions'][key], label=f'Binary Pred - {key}', alpha=0.7, color=color)

    plt.title(f'Predictions and Ground Truth over Time on {set_name}')
    plt.xlabel('Time (s)')
    plt.ylabel('Predictions')
    plt.legend()
    plt.xlim(25, 50)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=3)
    plt.savefig(f'plots/{set_name}.pdf',
            format='pdf', bbox_inches='tight')
    plt.close()

# Add coordinates to csv

In [ ]:
paths_json = os.path.join(os.getcwd(), 'settings/paths.json')
with open(paths_json, 'r') as json_file:
    paths = json.load(json_file)

model_root = f'/home/{os.getlogin()}/ERIE/realistic/Track-Shuyuan-2023-08-13/videos'
model_name = 'DLC_resnet50_TrackAug13shuffle1_70000'

training_sets = ["M1_L_V1_1", "M1_L_V1_2", "M1_R_V1_1", "M1_R_V1_2",
                 "M3_L_V1_1", "M3_L_V1_2", "M3_R_V1_1", "M3_R_V1_2",
                 "M5_L_V1_1", "M5_L_V1_2", "M5_R_V1_1", "M5_R_V1_2"]
test_sets = ["M1_L_V2_1", "M1_L_V2_2", "M1_R_V2_1", "M1_R_V2_2",
             "M3_L_V2_1", "M3_L_V2_2", "M3_R_V2_1", "M3_R_V2_2",
             "M5_L_V2_1", "M5_L_V2_2", "M5_R_V2_1", "M5_R_V2_2",
             "M2_L_V1_1", "M2_L_V1_2", "M2_R_V1_1", "M2_R_V1_2",
             "M4_L_V1_1", "M4_L_V1_2", "M4_R_V1_1", "M4_R_V1_2",
             "M2_L_V2_1", "M2_L_V2_2", "M2_R_V2_1", "M2_R_V2_2",
             "M4_L_V2_1", "M4_L_V2_2", "M4_R_V2_1", "M4_R_V2_2"]

data = pd.read_csv(paths['realistic']['labels'], header=0)

coordinates = {}

for dataset in training_sets + test_sets:
    coordinates[dataset] = pd.read_hdf(
        os.path.join(model_root, f'{dataset}_L_h264{model_name}.h5')).loc[:, [
            ('DLC_resnet50_TrackAug13shuffle1_70000', 'Mid_1', 'x'),
            ('DLC_resnet50_TrackAug13shuffle1_70000', 'Mid_1', 'y')]].to_numpy()
    
def extract_id(img_filename):
    return int(img_filename.split('_')[1].split('.')[0])

for index, row in data.iterrows():
    dataset = row['dataset']
    img_num = row['image']
    id = extract_id(img_num)
    
    x, y = coordinates[dataset][id]
    data.at[index, 'x'] = x
    data.at[index, 'y'] = y

data.to_csv(paths['realistic']['labels'], index=False)

# Bag Data Time Plots

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

test_set = ['C_M1_T1_8', 'M1_NF']


data_set = 'silicone'
for set_name in test_set:
    # concat paths
    label_path = os.path.join(
        f'/home/sxy841/ERIE/{data_set}/output', f'{set_name}', 'labels_30hz.txt')
    nn_label_path = os.path.join(
        f'/home/sxy841/ERIE/{data_set}/nn-pose/labels', f'{set_name}_gnn.txt')

    nn_positions = pd.read_csv(nn_label_path, header=None).to_numpy()
    positions = pd.read_csv(
        label_path, header=None).iloc[:, 7:10].to_numpy()
    desired_positions = pd.read_csv(
        label_path, header=None).iloc[:, 14:17].to_numpy()
    pos_diff = desired_positions-positions
    psm_force = pd.read_csv(
        label_path, header=None).iloc[:, 62:65].to_numpy()
    force = pd.read_csv(
        label_path, header=None).iloc[:, 1:4].to_numpy()
    
    gt_labels = []
    force_threshold = 0.2
    for row in force:
        gt_labels.append(True if np.sqrt(row.dot(row)) > force_threshold else False)

    # Plotting
    fig, axs = plt.subplots(3, figsize=(10, 15))
    
    print(set_name)

    # X, Y, Z components of positions
    for i in range(3):
        # axs[i].plot(positions[:, i],force[:,i], label='Actual Position')
        # axs[i].plot(positions[:, i],2000*pos_diff[:,i], '.' ,label='Actual Position')
        # axs[i].plot(1000*pos_diff[:,i], '.' ,label='PosDiff')
        axs[i].plot(positions[:, i], label='Actual Position')
        # axs[i].plot(force[:, i], label='Force')
        # axs[i].plot(psm_force[:, i], label='PSM Force')
        # axs[i].plot(nn_positions[:,i],label='NN Position')
        # axs[i].plot(desired_positions[:, i], label='Desired Position')
        axs[i].set_title(f'Component {i+1} (X, Y, Z respectively)')
        axs[i].legend()
        axs[i].set_xlim(0,1000)

    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

test_set = ['C_M1_T1_8', 'M1_NF']


data_set = 'silicone'
for set_name in test_set:
    # concat paths
    label_path = os.path.join(
        f'/home/sxy841/ERIE/{data_set}/output', f'{set_name}', 'labels_30hz.txt')
    nn_label_path = os.path.join(
        f'/home/sxy841/ERIE/{data_set}/nn-pose/labels', f'{set_name}_gnn.txt')

    nn_positions = pd.read_csv(nn_label_path, header=None).to_numpy()
    positions = pd.read_csv(
        label_path, header=None).iloc[:, 7:10].to_numpy()
    desired_positions = pd.read_csv(
        label_path, header=None).iloc[:, 14:17].to_numpy()
    pos_diff = desired_positions-positions
    psm_force = pd.read_csv(
        label_path, header=None).iloc[:, 62:65].to_numpy()
    force = pd.read_csv(
        label_path, header=None).iloc[:, 1:4].to_numpy()
    
    gt_labels = []
    force_threshold = 0.2
    for row in force:
        gt_labels.append(True if np.sqrt(row.dot(row)) > force_threshold else False)

    psm_force = MinMaxScaler().fit_transform(psm_force[gt_labels])
    positions = MinMaxScaler().fit_transform(positions[gt_labels])
    force = MinMaxScaler().fit_transform(force[gt_labels])
    

    # Plotting
    fig, axs = plt.subplots(3, figsize=(10, 15))
    
    print(set_name)

    # X, Y, Z components of positions
    for i in range(3):
        # axs[i].plot(positions[:, i],force[:,i], label='Actual Position')
        # axs[i].plot(positions[:, i],2000*pos_diff[:,i], '.' ,label='Actual Position')
        # axs[i].plot(1000*pos_diff[:,i], '.' ,label='PosDiff')
        axs[i].plot(positions[:, i], label='Actual Position')
        axs[i].plot(force[:, i], label='Force')
        # axs[i].plot(psm_force[:, i], label='PSM Force')
        # axs[i].plot(nn_positions[:,i],label='NN Position')
        # axs[i].plot(desired_positions[:, i], label='Desired Position')
        axs[i].set_title(f'Component {i+1} (X, Y, Z respectively)')
        axs[i].legend()
        axs[i].set_xlim(0,1000)

    plt.tight_layout()
    plt.show()